# AstroRV Catalog Ensambling

In [1]:
# =============================================================================
# Libraries
# =============================================================================
import pandas as pd
import numpy as np
import urllib.request
import collections
import os,re
import time
pd.options.mode.chained_assignment = None

# =============================================================================
# Data directories
# =============================================================================
dir_gaia = "src/Astro/"
dir_hip = "src/Astro/"
dir_simbad = "src/Astro/"
dir_rv = "src/RV/"
dir_results = "src/"

## =============================================================================
## Load Gaia database
## =============================================================================

In [2]:
t0 = time.time()
cols_gaia = ["source_id", "hip", "tycho2_id", "ra", "ra_error", "dec", "dec_error", "parallax", "parallax_error", 
             "pmra", "pmra_error", "pmdec", "pmdec_error", "ra_dec_corr", "ra_parallax_corr", "ra_pmra_corr", 
             "ra_pmdec_corr", "dec_parallax_corr", "dec_pmra_corr", "dec_pmdec_corr", "parallax_pmra_corr", 
             "parallax_pmdec_corr", "pmra_pmdec_corr", "phot_g_mean_flux", "phot_g_mean_flux_error", "phot_g_mean_mag", 
             "l", "b", "ecl_lon", "ecl_lat"]

gaia = pd.DataFrame()
for i in range(16):
    filename = dir_gaia + "TgasSource_000-000-0" + str(i).zfill(2) + ".csv.gz"
    gaia = gaia.append(pd.read_csv(filename, usecols=cols_gaia))

gaia.source_id = gaia.source_id.astype(str)
gaia.hip = gaia.hip.astype(str)
gaia.tycho2_id = gaia.tycho2_id.astype(str)

gaia["hip"] = gaia["hip"].map(lambda x: x.replace(".0", ""))
gaia.hip.replace("nan", "", inplace=True)
gaia.tycho2_id.replace("nan", "", regex=True, inplace=True)

n1 = len(gaia)
gaia = gaia[gaia.parallax > 0]
n2 = len(gaia)
print("\nDatabase Gaia-TGAS:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)

gaia_hip = gaia[gaia.hip != ""]
gaia_tyc = gaia[gaia.tycho2_id != ""]


Database Gaia-TGAS: 
Original objects: 2057050 
Discarded objects: 30840 
Final objects: 2026210


## =============================================================================
## Load Hipparcos database
## =============================================================================

In [3]:
# Available in: http://cdsarc.u-strasbg.fr/viz-bin/Cat?cat=I%2F239&target=readme&#sRM2.1
names_hip={}
names_hip[1] = "hip"
names_hip[8] = "ra_hip"
names_hip[9] = "dec_hip"
names_hip[14] = "ra_error_hip"
names_hip[15] = "dec_error_hip"
names_hip[19] = "ra_dec_corr_hip"
names_hip[20] = "ra_parallax_corr_hip"
names_hip[21] = "dec_parallax_corr_hip"
names_hip[22] = "ra_pmra_corr_hip"
names_hip[23] = "dec_pmra_corr_hip"
names_hip[24] = "parallax_pmra_corr_hip"
names_hip[25] = "ra_pmdec_corr_hip"
names_hip[26] = "dec_pmdec_corr_hip"
names_hip[27] = "parallax_pmdec_corr_hip"
names_hip[28] = "pmra_pmdec_corr_hip"
names_hip[5] = "Vmag_hip"
names_hip[71] = "HenryDraperId_hip"
#The following columns are not read and left to the improved Hipparcos catalogue:
# names_hip[11] = "parallax_hip"
# names_hip[12] = "pmra_hip"
# names_hip[13] = "pmdec_hip"
# names_hip[16] = "parallax_error_hip"
# names_hip[17] = "pmra_error_hip"
# names_hip[18] = "pmdec_error_hip"
#names2_hip[76] = "sptype_hip"

names_hip = collections.OrderedDict(sorted(names_hip.items()))

hipparcos = pd.read_csv(dir_hip + "hip_main.dat.gz", delimiter="|", usecols=names_hip.keys(), names=names_hip.values())

# Columns format
cols = hipparcos.columns.tolist()
objects = ["hip", "HenryDraperId_hip"]
cols = [x for x in cols if x not in objects]
hipparcos[cols] = hipparcos[cols].apply(pd.to_numeric, errors='coerce')

hipparcos.hip = hipparcos.hip.astype(str)
hipparcos.hip = hipparcos.hip.map(lambda x: x.replace(".0", ""))

# Delete null astrometry values
n1 = len(hipparcos)
hipparcos.dropna(subset=["ra_hip", "dec_hip"], how="any", inplace=True)
n2 = len(hipparcos)
print("\nDatabase Hipparcos:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)

/home/iwander/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



Database Hipparcos: 
Original objects: 118218 
Discarded objects: 263 
Final objects: 117955


## =============================================================================
## Load Hipparcos improved
## =============================================================================

In [16]:
comments=list(range(95))+[96,97]
hipvan=pd.read_csv(dir_hip + "HipVanLeeuwen2007.tsv",sep=";",skiprows=comments)
hipvan=hipvan[['_RAJ2000', '_DEJ2000', 'HIP', 'Plx', 'e_Plx', 'pmRA', 'e_pmRA',
       'pmDE', 'e_pmDE','Hpmag', 'e_Hpmag', 'B-V', 'e_B-V', 'V-I']]
columns={'_RAJ2000':'ra_hip', '_DEJ2000':'dec_hip', 'HIP':'hip', 
         'Plx':'parallax_hip', 'e_Plx':'parallax_error_hip', 
         'pmRA':'pmra_hip', 'e_pmRA':'pmra_error_hip',
         'pmDE':'pmdec_hip', 'e_pmDE':'pmdec_error_hip','Hpmag':'Hpmag_hip', 
         'e_Hpmag':'e_Hpmag_hip', 'B-V':'B_V_hip', 'e_B-V':'e_B_V_hip',
         'V-I':'V_I_hip'}
hipvan.rename(columns=columns,inplace=True)

cols = hipvan.columns.tolist()
objects = ["hip"]
cols = [x for x in cols if x not in objects]
hipvan[cols] = hipvan[cols].apply(pd.to_numeric, errors='coerce')

hipvan.hip = hipvan.hip.astype(str)
hipvan.hip = hipvan.hip.map(lambda x: x.replace(".0", ""))

hipvan=hipvan[hipvan.parallax_hip > 0]

# Delete null astrometry values
n1 = len(hipvan)
hipvan.dropna(subset=["ra_hip", "dec_hip", "parallax_hip"], how="any", inplace=True)
n2 = len(hipvan)
print("\nDatabase Hipparcos improved:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)


Database Hipparcos improved: 
Original objects: 113942 
Discarded objects: 0 
Final objects: 113942


In [19]:
# Merge Hipparcos + HipVanLeeuwen2007
cols = ['hip', 'parallax_hip', 'pmra_hip', 'pmdec_hip', 'parallax_error_hip', 'pmra_error_hip','pmdec_error_hip']
hipparcos = hipparcos.merge(hipvan[cols], on='hip', how='left')

## =============================================================================
## Load Tycho database
## =============================================================================

In [22]:
# Available in: http://cdsarc.u-strasbg.fr/viz-bin/Cat?cat=I%2F239&target=readme&#sRM2.13

names_tyc={}
names_tyc[1] = "tycho2_id"
names_tyc[8] = "ra_tyc"
names_tyc[9] = "dec_tyc"
names_tyc[11] = "parallax_tyc"
names_tyc[12] = "pmra_tyc"
names_tyc[13] = "pmdec_tyc"
names_tyc[14] = "ra_error_tyc"
names_tyc[15] = "dec_error_tyc"
names_tyc[16] = "parallax_error_tyc"
names_tyc[17] = "pmra_error_tyc"
names_tyc[18] = "pmdec_error_tyc"
names_tyc[19] = "ra_dec_corr_tyc"
names_tyc[20] = "ra_parallax_corr_tyc"
names_tyc[21] = "dec_parallax_corr_tyc"
names_tyc[22] = "ra_pmra_corr_tyc"
names_tyc[23] = "dec_pmra_corr_tyc"
names_tyc[24] = "parallax_pmra_corr_tyc"
names_tyc[25] = "ra_pmdec_corr_tyc"
names_tyc[26] = "dec_pmdec_corr_tyc"
names_tyc[27] = "parallax_pmdec_corr_tyc"
names_tyc[28] = "pmra_pmdec_corr_tyc"
names_tyc[53] = "HenryDraperId_tyc"
names_tyc[5] = "Vmag_tyc"

names_tyc = collections.OrderedDict(sorted(names_tyc.items()))

tycho = pd.read_csv(dir_hip + "tyc_main.zip", delimiter="|", usecols = names_tyc.keys(), names = names_tyc.values())

# Split original tycho-id header which separated by white-space
tycho["a"], tycho["b"], tycho["c"] = tycho["tycho2_id"].str.split().str

# Concatenate tycho-id headers using "-"
tycho["tycho2_id"] = tycho["a"] + "-" + tycho["b"] + "-" + tycho["c"]

# Delete auxiliar columns used in conversion
del tycho["a"], tycho["b"], tycho["c"]

# Columns format
cols = tycho.columns.tolist()
objects = ["tycho2_id", "HenryDraperId_tyc"]
cols = [x for x in cols if x not in objects]
tycho[cols] = tycho[cols].apply(pd.to_numeric, errors='coerce')

tycho=tycho[tycho.parallax_tyc > 0]

# Delete null astrometry values
n1 = len(tycho)
tycho.dropna(subset=["ra_tyc", "dec_tyc", "parallax_tyc"], how="any", inplace=True)
n2 = len(tycho)
print("\nDatabase Tycho:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)


Database Tycho: 
Original objects: 579054 
Discarded objects: 0 
Final objects: 579054


## =============================================================================
## Load Simbad-Hipparcos database
## =============================================================================

In [23]:
cols = ["typedident","identifier", "radvel", "coord1(ICRS,J2000/2000)", "plx", "pm", "MagV", "spec.type"]
simbad = pd.read_csv(dir_simbad + "simbad.zip", usecols=cols, delimiter="|")

# Modify ID
simbad["hip"] = simbad["typedident"].map(lambda x: str(x)[4:]).astype(str)
simbad["hip"] = simbad["hip"].str.strip()
del simbad["typedident"]

# Right ascension and declination format
# 1. Delete white-spaces on both sides of the text
simbad["coord1(ICRS,J2000/2000)"] = simbad["coord1(ICRS,J2000/2000)"].str.strip()

# 2. Split string into 6 values (hh:mm:ss for RA and hh:mm:ss for DEC)
simbad["ra_h"], simbad["ra_m"], simbad["ra_s"], simbad["dec_h"], simbad["dec_m"], simbad["dec_s"] = \
    simbad["coord1(ICRS,J2000/2000)"].str.split(" ").str

# 3. Concatenate the first 3 fields (RA) using conversion formule from hh:mm:ss to degrees
simbad["ra_simbad"] = simbad["ra_h"].astype(float)*15 + simbad["ra_m"].astype(float)/60 + simbad["ra_s"].astype(float)/3600

# 4. Concatenate the last 3 fields (DEC) using conversion formule from hh:mm:ss to degrees
simbad["dec_simbad"] = np.sign(simbad["dec_h"].astype(float)) * ( \
    np.abs(simbad["dec_h"].astype(float)) + simbad["dec_m"].astype(float)/60 + simbad["dec_s"].astype(float)/3600 )

# 5. Delete auxiliar columns used in conversion
del simbad["coord1(ICRS,J2000/2000)"]
del simbad["ra_h"], simbad["ra_m"], simbad["ra_s"], simbad["dec_h"], simbad["dec_m"], simbad["dec_s"]

# Proper motion format
# 1. Delete white-spaces on both sides of the text
simbad["pm"] = simbad["pm"].str.strip()

# 2. Split string into 2 values (pm_ra and pm_dec)
simbad["pmra_simbad"], simbad["pmdec_simbad"] = simbad["pm"].str.split(" ").str

# 3. Delete auxiliar columns used in conversion
del simbad["pm"]

# Modify name columns
simbad = simbad.rename(columns={"identifier": "name_simbad"})
simbad = simbad.rename(columns={"plx": "parallax_simbad"})   
simbad = simbad.rename(columns={"spec.type": "sptype_simbad"})     
simbad = simbad.rename(columns={"radvel": "RV_simbad"})    
simbad = simbad.rename(columns={"MagV": "Vmag_simbad"})

# Columns format
cols = simbad.columns.tolist()
objects = ["hip", "name_simbad", "sptype_simbad"]
cols = [x for x in cols if x not in objects]
simbad[cols] = simbad[cols].apply(pd.to_numeric, errors='coerce')

# Delete null astrometry values
n1 = len(simbad)
simbad.dropna(subset=["ra_simbad", "dec_simbad", "parallax_simbad"], how="any", inplace=True)      
n2 = len(simbad)

print("\nDatabase Simbad:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)



Database Simbad: 
Original objects: 118179 
Discarded objects: 175 
Final objects: 118004


## =============================================================================
## Create database with unique astrometry values
## =============================================================================

In [24]:
print("\nBuilding database with just one astrometry source per star:")

# db = Gaia + Hipparcos
db = pd.DataFrame()
gaia["source_astro"] = "gaia"
exclusive_hip = hipparcos[~hipparcos.hip.isin(gaia_hip.hip)]
exclusive_hip.rename(columns=lambda x: x.replace("_hip", ""), inplace=True)
exclusive_hip["source_astro"] = "hipparcos"
db = pd.concat([gaia, exclusive_hip], axis=0)
print("Input from Hipparcos:", len(exclusive_hip))

# db = db + Tycho
exclusive_tyc = tycho[~tycho.tycho2_id.isin(gaia_tyc.tycho2_id)]
exclusive_tyc.rename(columns=lambda x: x.replace("_tyc", ""), inplace=True)
exclusive_tyc["source_astro"] = "tycho"
db = pd.concat([db, exclusive_tyc], axis=0)
print("Input from Tycho:", len(exclusive_tyc))

# db = db + Simbad
exclusive_simbad = simbad[~simbad.hip.isin(db.hip)]
exclusive_simbad.rename(columns=lambda x: x.replace("_simbad", ""), inplace=True)
exclusive_simbad = exclusive_simbad.loc[:,["hip", "ra", "dec", "parallax", "pmra", "pmdec"]]
exclusive_simbad["source_astro"] = "simbad"
db = pd.concat([db, exclusive_simbad], axis=0)
print("Input from Simbad:", len(exclusive_simbad))

# Clean np.NaN in objects produced by concat method
db.replace({"hip":{np.NaN:""},"tycho2_id":{np.NaN:""},"source_id":{np.NaN:""},"HenryDraperId":{np.NaN:""}}, inplace=True)

# Merge db + Simbad new columns
db = db.merge(simbad[["hip", "name_simbad","sptype_simbad","RV_simbad","Vmag_simbad"]], on="hip", how="left")

# Merge db + HipVanLeeuwen2007 new columns
db = db.merge(hipvan[['hip', 'Hpmag_hip', 'e_Hpmag_hip', 'B_V_hip', 'e_B_V_hip', 'V_I_hip']], on='hip', how='left')

#db["id"] = db.hip.combine(db.tycho2_id, lambda x, y: x if x is not "" else y)
db["id"] = db.hip.astype(str) + db.tycho2_id.astype(str)

# Change the order of database columns
cols = ["id", "source_astro"] + cols_gaia + ["Vmag","HenryDraperId"] + \
       ["name_simbad","sptype_simbad","RV_simbad","Vmag_simbad"] + \
       ['Hpmag_hip', 'e_Hpmag_hip', 'B_V_hip', 'e_B_V_hip', 'V_I_hip']
db = db[cols]

#Remove all stars without astrometric errors
errors=["ra_error","dec_error","parallax_error","pmra_error","pmdec_error"]
db.dropna(subset=errors,how="any",inplace=True)

db = db.reset_index(drop=True)

print("Total lenght:", len(db))


Building database with just one astrometry source per star:
Input from Hipparcos: 24557
Input from Tycho: 164550
Input from Simbad: 67
Total lenght: 2214419


# =============================================================================
# Full database with the information of all the catalogs
# =============================================================================

In [25]:
print("\nMerging astrometry databases (it keeps data of all sources):")

database = pd.DataFrame()

n1 = len(gaia)
print("Gaia objects (initial lenght of database):", n1)

database = gaia.merge(hipparcos, on="hip", how="outer")
n2 = len(database)
print("Hipparcos objects:", len(hipparcos), "| Input from Hipparcos:", n2-n1, "| Final lenght of database:", n2)

database = database.merge(hipvan, on="hip", how="outer")
n21 = len(database)
print("Hipparcos improved objects:", len(hipvan), "| Input from Hipparcos improved:", n21-n2, "| Final lenght of database:", n21)

database = database.merge(tycho, on="tycho2_id", how="outer")
n3 = len(database)
print("Tycho objects:", len(tycho), "| Input from Tycho:", n3-n2, "| Final lenght of database:", n3)

database = database.merge(simbad, on="hip", how="outer")
n4 = len(database)
print("Simbad objects:", len(simbad),  "| Input from Simbad:", n4-n3, "| Final lenght of database:", n4)

#database["id"] = database.hip.combine(database.tycho2_id, lambda x, y: x if x is not "" else y)
database["id"] = database.hip.astype(str) + database.tycho2_id.astype(str)

#Remove all stars without astrometric errors
database.dropna(subset=["ra_error","dec_error","parallax_error","pmra_error","pmdec_error"],how="any",inplace=True)

database = database.reset_index(drop=True)


Merging astrometry databases (it keeps data of all sources):
Gaia objects (initial lenght of database): 2026210
Hipparcos objects: 117955 | Input from Hipparcos: 24557 | Final lenght of database: 2050767
Hipparcos improved objects: 113942 | Input from Hipparcos improved: 0 | Final lenght of database: 2050767
Tycho objects: 579054 | Input from Tycho: 164550 | Final lenght of database: 2215317
Simbad objects: 118004 | Input from Simbad: 67 | Final lenght of database: 2215384


# =============================================================================
# Load radial velocities databases
# =============================================================================

In [26]:
t1 = time.time()
print("\nRadial velocity databases:")
radialv = {}


Radial velocity databases:


## RAVE-DR5

In [27]:
# =============================================================================
# RAVE-DR5
# =============================================================================

file = "RAVE-DR5.tsv"
cols = ["HRV", "e_HRV", "TGAS", "TYCHO2"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                                skiprows = list(np.arange(78)) + [79, 80]))

data.rename(columns={"HRV": "RV", "e_HRV": "e_RV", "TGAS": "source_id", "TYCHO2": "tycho2_id"}, inplace=True)
data = data.merge(gaia.loc[:,["source_id","hip"]], left_on="source_id", right_on="source_id", how="left")
data.source_id = data.source_id.str.strip()
data.tycho2_id = data.tycho2_id.str.strip()
data.hip.replace(np.nan,"", inplace=True)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 RAVE-DR5.tsv 
Original objects: 520701 
Discarded: 0 
Final objects: 520701


## BB2000

In [75]:
# =============================================================================
# BB2000
# =============================================================================

file = "BB2000.csv"
cols = ["RV", "e_RV", "TYC1", "TYC2", "TYC3"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data["tycho2_id"] = data.TYC1.astype(str) + "-" + data.TYC2.astype(str) + "-" + data.TYC3.astype(str)
del data["TYC1"], data["TYC2"], data["TYC3"]

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()

data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median

n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 BB2000.csv 
Original objects: 673 
Discarded: 0 
Final objects: 673


## Famaey2005.tsv

In [29]:
# =============================================================================
# Famaey2005.tsv
# =============================================================================

file = "Famaey2005.tsv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                                skiprows = list(np.arange(118)) + [119, 120]))

data.rename(columns={"HIP": "hip"}, inplace=True)
data.hip = data.hip.astype(str)
data.RV = pd.to_numeric(data.RV, errors="coerce")

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 Famaey2005.tsv 
Original objects: 6690 
Discarded: 662 
Final objects: 6028


## Galah.tsv - Without e_RV

In [30]:
# =============================================================================
# Galah.tsv - Without e_RV
# =============================================================================

file = "Galah.tsv"
cols = ["RV", "TYC2"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                                skiprows = list(np.arange(54)) + [55, 56]))

data.rename(columns={"TYC2": "tycho2_id"}, inplace=True)
data["tycho2_id"] = data["tycho2_id"].map(lambda x: x.replace("-", " "))
data["a"], data["b"], data["c"] = data["tycho2_id"].str.split().str
data["a"] = pd.to_numeric(data["a"])
data["b"] = pd.to_numeric(data["b"])
data["c"] = pd.to_numeric(data["c"])
data["tycho2_id"] = data["a"].astype(str) + "-" + data["b"].astype(str) + "-" + data["c"].astype(str)
del data["a"], data["b"], data["c"]

data["source_rv"] = file
data["source_erv"] = "Without_info"
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)

data["e_RV"] = 0.6 

n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 Galah.tsv 
Original objects: 10680 
Discarded: 0 
Final objects: 10680


## GCS2011.tsv

In [31]:
# =============================================================================
# GCS2011.tsv
# =============================================================================

file = "GCS2011.tsv"
cols = ["RV", "e_RV", "HIP", "Name"]
delimiter = "|"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                               skiprows = list(np.arange(174)) + [175, 176]))

data.rename(columns={"HIP":"hip", "Name":"name"}, inplace=True)
data.RV = pd.to_numeric(data.RV, errors="coerce")
data.e_RV = pd.to_numeric(data.e_RV, errors="coerce")
data.hip = data.hip.str.strip()
data.name = data.name.str.strip()

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 GCS2011.tsv 
Original objects: 16682 
Discarded: 2543 
Final objects: 14139


## Malaroda2012.csv - Without e_RV

In [32]:
# =============================================================================
# Malaroda2012.csv - Without e_RV
# =============================================================================

file = "Malaroda2012.csv"
cols = ["RV", "HIP", "TYC1", "TYC2", "TYC3"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data.rename(columns={"HIP":"hip"}, inplace=True)

data["tycho2_id"] = data.TYC1.astype(str) + "-" + data.TYC2.astype(str) + "-" + data.TYC3.astype(str)
del data["TYC1"], data["TYC2"], data["TYC3"]

data["e_RV"]=1.0
data["source_rv"] = file
data["source_erv"] = "Without_info"
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 Malaroda2012.csv 
Original objects: 2178 
Discarded: 146 
Final objects: 2032


## Maldonado2010.tsv

In [33]:
# =============================================================================
# Maldonado2010.tsv
# =============================================================================

file = "Maldonado2010.tsv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                               skiprows = list(np.arange(82)) + [83, 84]))

data.rename(columns={"HIP":"hip"}, inplace=True)
data.RV = pd.to_numeric(data.RV, errors="coerce")
data.e_RV = pd.to_numeric(data.e_RV, errors="coerce")
data.hip = data.hip.astype(str)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 Maldonado2010.tsv 
Original objects: 495 
Discarded: 22 
Final objects: 473


## Pulkovo.tsv

In [34]:
# =============================================================================
# Pulkovo.tsv
# =============================================================================

file = "Pulkovo.tsv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                               skiprows = list(np.arange(61)) + [62, 63]))

data.rename(columns={"HIP":"hip"}, inplace=True)
data.hip = data.hip.astype(str)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"

median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median

n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)



 Pulkovo.tsv 
Original objects: 35493 
Discarded: 0 
Final objects: 35493


## Web1995-HIP.csv

In [35]:
# =============================================================================
# Web1995-HIP.csv
# =============================================================================

file = "Web1995-HIP.csv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data.rename(columns={"HIP":"hip"}, inplace=True)
data.hip = data.hip.astype(str)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()

data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median

n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 Web1995-HIP.csv 
Original objects: 494 
Discarded: 0 
Final objects: 494


## Web1995-TYC2.csv

In [36]:
# =============================================================================
# Web1995-TYC2.csv
# =============================================================================

file = "Web1995-TYC2.csv"
cols = ["RV", "e_RV", "TYC1", "TYC2", "TYC3"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data["tycho2_id"] = data.TYC1.astype(str) + "-" + data.TYC2.astype(str) + "-" + data.TYC3.astype(str)
del data["TYC1"], data["TYC2"], data["TYC3"]

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
data.e_RV[data.e_RV==0] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)



 Web1995-TYC2.csv 
Original objects: 673 
Discarded: 0 
Final objects: 673


## APOGEE2 (taken from RVcat of Bailer-Jones 2017)

In [37]:
# =============================================================================
# RVcat.csv.zip
# =============================================================================
file = "RVcat.csv.zip"
cols = ["RV", "RVerr", "tychoID","cat"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

#Choose only objects from apogee database
data = data[data.cat==10]

data.rename(columns={"RVerr":"e_RV", "tychoID":"tycho2_id"}, inplace=True)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
n1 = len(data)
data.dropna(subset=["e_RV"], inplace=True)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)


 RVcat.csv.zip 
Original objects: 29173 
Discarded: 0 
Final objects: 29173


## =============================================================================
## Radial velocity databases integration
## =============================================================================

In [38]:
print("\nRadial velocity databases integration:")

t2 = time.time()
RV = pd.DataFrame()
RV = pd.concat([radialv["RAVE-DR5.tsv"], radialv["BB2000.csv"], radialv["Famaey2005.tsv"], 
                radialv["Galah.tsv"], radialv["GCS2011.tsv"], radialv["Malaroda2012.csv"], 
                radialv["Maldonado2010.tsv"], radialv["Pulkovo.tsv"], radialv["Web1995-HIP.csv"], 
                radialv["Web1995-TYC2.csv"], radialv["RVcat.csv.zip"]], axis = 0)
RV.hip.replace(np.NaN, "", inplace=True)
RV.tycho2_id.replace(np.NaN, "", inplace=True)
RV.source_id.replace(np.NaN, "", inplace=True)
RV.name.replace(np.NaN, "", inplace=True)
RV["id"] = RV.hip.astype(str) + RV.tycho2_id.astype(str)

# Drop objects with incomplete astrometry information or with both RV and e_RV duplicates
RV1 = RV.copy()
RV1 = RV1[RV1.e_RV.notnull()]
RV1 = RV1[RV1.id != ""]

# RV1 => Various records per star with different VR and e_VR combination are allowed
RV1.drop_duplicates(subset=["id","RV","e_RV"], keep="first", inplace=True)

# RV2 => Keep only object with minimum e_RV value (various records per star with the same e_VR are allowed)
RV2 = pd.DataFrame()
RV_min_eRV = RV1.groupby("id", as_index=False).e_RV.min()
RV2 = RV1.merge(RV_min_eRV, on=["id","e_RV"], how="inner")

# RV3 => Keep only one object with minimum e_RV value (Just one record per star is allowed)
RV3 = pd.DataFrame()
RV3 = RV2.drop_duplicates(subset=["id","e_RV"], keep="first")

print("Objects number with different VR and e_VR combination:", len(RV1))
print("Objects number with different VR and the same minimun e_VR:", len(RV2))
print("Objects number without duplicates (just one register per star):", len(RV3))

t3 = time.time()


Radial velocity databases integration:
Objects number with different VR and e_VR combination: 407189
Objects number with different VR and the same minimun e_VR: 335367
Objects number without duplicates (just one register per star): 332551


## =============================================================================
## Full integration
## =============================================================================

In [39]:
# Building AstroRV's databases
AstroRVDuplicates = pd.DataFrame()
AstroRVDuplicates = db.merge(RV1[["id", "RV", "e_RV", "source_rv", "source_erv"]], on="id", how="right")

AstroRV = pd.DataFrame()
AstroRV = db.merge(RV3[["id", "RV", "e_RV", "source_rv", "source_erv"]], on="id", how="right")

# Drop objects with incomplet astrometry information
AstroRVDuplicates.dropna(subset=["parallax"], inplace=True)
AstroRV.dropna(subset=["parallax"], inplace=True)

# Remove spaces from simbad names
for txt_field in "name_simbad","sptype_simbad","HenryDraperId":
    AstroRV[txt_field]=AstroRV[txt_field].fillna('')
    AstroRV[txt_field]=AstroRV[txt_field].map(lambda x:'NULL' if x=='' else re.sub('[\s\",]+','_',x).strip('_'))

    AstroRVDuplicates[txt_field]=AstroRVDuplicates[txt_field].fillna('')
    AstroRVDuplicates[txt_field]=AstroRVDuplicates[txt_field].map(lambda x:'NULL' if x=='' else re.sub('[\s\",]+','_',x).strip('_'))

# Report
print("\nObjects in AstroRV (without duplicates):", len(AstroRV))
print("\nObjects in AstroRV per source:", AstroRV.groupby(AstroRV.source_rv).source_rv.count())

print("Number of objects with hipparcos id:",(AstroRV.hip!="").sum())

report=dict()
print("Number of objects with tycho2 id:",(AstroRV.tycho2_id!="").sum())


Objects in AstroRV (without duplicates): 285114

Objects in AstroRV per source: source_rv
BB2000.csv              503
Famaey2005.tsv         5544
GCS2011.tsv            7091
Galah.tsv              7837
Malaroda2012.csv        416
Maldonado2010.tsv       301
Pulkovo.tsv           23412
RAVE-DR5.tsv         217257
RVcat.csv.zip         22501
Web1995-HIP.csv         252
Name: source_rv, dtype: int64
Number of objects with hipparcos id: 36699
Number of objects with tycho2 id: 248415


In [ ]:
#Save databases
AstroRVDuplicates.to_csv(dir_results + "AstroRVFull.csv", index=False)
AstroRV.to_csv(dir_results + "AstroRV.csv", index=False)

## =============================================================================
## Summaries
## =============================================================================

In [158]:
report=dict()

print("GAIA TGAS:")
rep=report["Gaia TGAS"]=dict()
rep["ref"]="I/337/tgas & (1)"
rep["total"]=len(gaia)
rep["hip"]=(gaia.hip!="").sum()
rep["tyc"]=(gaia.tycho2_id!="").sum()
rep["cont"]=len(gaia)
print("\tNumber of objects:",rep["total"])
print("\tHipparcos:",rep["hip"])
print("\tTycho:",rep["tyc"])
print("\tContribution:",rep["cont"])

print("Hipparcos:")
rep=report["Hipparcos"]=dict()
rep["ref"]="I/239/hip\_main & (2)"
rep["total"]=len(hipparcos)
rep["hip"]=(hipparcos.hip!="").sum()
rep["tyc"]=0
rep["cont"]=len(exclusive_hip)
print("\tNumber of objects:",rep["total"])
print("\tHipparcos:",rep["hip"])
print("\tTycho:",rep["tyc"])
print("\tContribution:",rep["cont"])

print("Tycho:")
rep=report["Tycho"]=dict()
rep["ref"]="I/259/tyc2 & (2)"
rep["total"]=len(tycho)
rep["hip"]=0
rep["tyc"]=(tycho.tycho2_id!="").sum()
rep["cont"]=len(exclusive_tyc)
print("\tNumber of objects:",rep["total"])
print("\tHipparcos:",rep["hip"])
print("\tTycho:",rep["tyc"])
print("\tContribution:",rep["cont"])

print("Simbad:")
rep=report["Simbad"]=dict()
rep["ref"]="-- & (3)"
rep["total"]=len(simbad)
rep["hip"]=(simbad.hip!="").sum()
rep["tyc"]=0
rep["cont"]=len(exclusive_simbad)
print("\tNumber of objects:",rep["total"])
print("\tHipparcos:",rep["hip"])
print("\tTycho:",rep["tyc"])
print("\tContribution:",rep["cont"])

astro_cats=["Gaia TGAS","Hipparcos","Tycho","Simbad"]
report["astro_total"]=0
report["astro_hip"]=0
report["astro_tyc"]=0
report["astro_cont"]=0
for cat in astro_cats:report["astro_total"]+=report[cat]["total"]
for cat in astro_cats:report["astro_hip"]+=report[cat]["hip"]
for cat in astro_cats:report["astro_tyc"]+=report[cat]["tyc"]
for cat in astro_cats:report["astro_cont"]+=report[cat]["cont"]
print("Summary:")    
print("\tTotal:",report["astro_total"])
print("\tHipparcos:",report["astro_hip"])
print("\tTycho:",report["astro_tyc"])
print("\tCatalog:",report["astro_cont"])

rv_cats=collections.OrderedDict([
    ["WEB1995",dict(cat=["Web1995-TYC2.csv","Web1995-HIP.csv"],ref=r"III/213 & (5)")],
    ["GCS",dict(cat=["GCS2011.tsv"],ref=r"J/A+A/530/A138 & (6)")],
    ["RAVE-DR5",dict(cat=["RAVE-DR5.tsv"],ref=r"III/279/rave\_dr5 & (7)")],
    ["PULKOVO",dict(cat=["Pulkovo.tsv"],ref=r"III/252/table8 & (8)")],
    ["FAMAEY2005",dict(cat=["Famaey2005.tsv"],ref=r"J/A+A/430/165/tablea1 & (9)")],
    ["BB2000",dict(cat=["BB2000.csv"],ref=r"III/213 & (10)")],
    ["MALARODA",dict(cat=["Malaroda2012.csv"],ref=r"III/249/catalog & (11)")],
    ["GALAH",dict(cat=["Galah.tsv"],ref=r"J/MNRAS/465/3203 & (12)")],
    ["MALDONADO",dict(cat=["Maldonado2010.tsv"],ref=r"J/A+A/521/A12/table1 & (13)")],
    ["APOGEE2",dict(cat=["RVcat.csv.zip"],ref="-- & (14)")]
    ])

i=0
for key in rv_cats.keys():
    cats=rv_cats[key]["cat"]
    print("Catalog ",key)
    report[key]=dict()
    for k in "total","hip","tyc","cont":report[key][k]=0
    for cat in cats:
        print("\tCatalog %s..."%cat)
        report[key]["total"]+=len(radialv[cat])
        try:report[key]["hip"]+=(radialv[cat].hip!="").sum()
        except:report[key]["hip"]+=0
        try:report[key]["tyc"]+=(radialv[cat].tycho2_id!="").sum()
        except:report[key]["tyc"]+=0
        report[key]["cont"]+=(AstroRV.source_rv==cat).sum()
    print("\tTotal:",report[key]["total"])
    print("\tHipparcos:",report[key]["hip"])
    print("\tTycho:",report[key]["tyc"])
    print("\tContribution:",report[key]["cont"])
    i+=1

report["rv_total"]=0
report["rv_hip"]=0
report["rv_tyc"]=0
report["rv_cont"]=0
cats=rv_cats.keys()
for cat in cats:report["rv_total"]+=report[cat]["total"]
for cat in cats:report["rv_hip"]+=report[cat]["hip"]
for cat in cats:report["rv_tyc"]+=report[cat]["tyc"]
for cat in cats:report["rv_cont"]+=report[cat]["cont"]
print("Summary radial velocities:")    
print("\tTotal:",report["rv_total"])
print("\tHipparcos:",report["rv_hip"])
print("\tTycho:",report["rv_tyc"])
print("\tCatalog:",report["rv_cont"])

GAIA TGAS:
	Number of objects: 2026210
	Hipparcos: 93398
	Tycho: 1932812
	Contribution: 2026210
Hipparcos:
	Number of objects: 117955
	Hipparcos: 117955
	Tycho: 0
	Contribution: 24557
Tycho:
	Number of objects: 579054
	Hipparcos: 0
	Tycho: 579054
	Contribution: 164550
Simbad:
	Number of objects: 118004
	Hipparcos: 118004
	Tycho: 0
	Contribution: 67
Summary:
	Total: 2841223
	Hipparcos: 329357
	Tycho: 2511866
	Catalog: 2215384
Catalog  WEB1995
	Catalog Web1995-TYC2.csv...
	Catalog Web1995-HIP.csv...
	Total: 1167
	Hipparcos: 494
	Tycho: 673
	Contribution: 252
Catalog  GCS
	Catalog GCS2011.tsv...
	Total: 14139
	Hipparcos: 12977
	Tycho: 0
	Contribution: 7091
Catalog  RAVE-DR5
	Catalog RAVE-DR5.tsv...
	Total: 520701
	Hipparcos: 121
	Tycho: 309596
	Contribution: 217257
Catalog  PULKOVO
	Catalog Pulkovo.tsv...
	Total: 35493
	Hipparcos: 35493
	Tycho: 0
	Contribution: 23412
Catalog  FAMAEY2005
	Catalog Famaey2005.tsv...
	Total: 6028
	Hipparcos: 6028
	Tycho: 0
	Contribution: 5544
Catalog  BB2000


/home/iwander/.local/lib/python3.5/site-packages/pandas/core/ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [165]:
f=open(dir_results+"AstroRVSummary.tex","w")

#Header
f.write(r"""\begin{table*}
\centering
\scriptsize
\begin{tabular}{lllllll}
\hline 
Catalog name & Number of objects & Hipparcos ID & Tycho-2 ID & Contribution & CDS Code & Reference \\\hline\hline
\multicolumn{7}{c}{\tt Astrometric} \\
\hline

""")

#Astrometry
for cat in astro_cats:
    hip=str(report[cat]["hip"]) if report[cat]["hip"]>0 else "--"
    tyc=str(report[cat]["tyc"]) if report[cat]["tyc"]>0 else "--"
    f.write(r"""%s & %d & %s & %s & %d & \tt %s \\\

"""%(cat,report[cat]["total"],hip,tyc,report[cat]["cont"],report[cat]["ref"]))

f.write(r"""
  \hline
  {\tt Totals} & \bf %d & \bf %d & \bf %d & \bf %d & This work \\
  \hline
"""%(report["astro_total"],report["astro_hip"],report["astro_tyc"],report["astro_cont"]))

#Radial Velocity catalogues
f.write(r"""
  \multicolumn{7}{c}{\tt Radial velocities} \\
  \hline
""")

for cat in rv_cats.keys():
    hip=str(report[cat]["hip"]) if report[cat]["hip"]>0 else "--"
    tyc=str(report[cat]["tyc"]) if report[cat]["tyc"]>0 else "--"
    f.write(r"""%s & %d & %s & %s & %d & \tt %s \\\

"""%(cat,report[cat]["total"],hip,tyc,report[cat]["cont"],rv_cats[cat]["ref"]))

f.write(r"""
  \hline
  {\tt AstroRV} & \bf %d & \bf %d & \bf %d & \bf %d & This work \\
  \hline
"""%(report["rv_total"],report["rv_hip"],report["rv_tyc"],report["rv_cont"]))

#Footer
f.write(r"""
\end{tabular}
\caption{Catalogs used to compile the {\tt AstroRV} catalog for this work. References: (1) \citealt{Brown2016}, (2) \citealt{ESA1997} (3) \citealt{Wenger2000}, (5) \citealt{Barbier2000a}, (6) \citealt{Casagrande2011}, (7) \citealt{Kunder2017}, (8)  \citealt{Gontcharov2006}, (9) \citealt{Famaey2005}, (10) \citealt{Barbier2000b}, (11) \citealt{Malaroda2000}, (12) \citealt{Martell2016}, (13) \citealt{Maldonado2010}, (14) \citealt{Bailer2017}.
\label{tab:AstroRV}}
\end{table*}
""")
f.close()

print("Done.")

Done.
